# Duckietown NCTU - Tutorial 3: Finite State Machine

By Chang-Yi Kuo, Brian Chuang, and Nick Wang

Before you run this jupyter notebook on your duckietop, make sure you 

```sh
duckietop $ roslaunch duckietown_nctu_wama joystick_jupyter.launch veh:=duckiebot
duckietop $ source ~/duckietown/environment.sh
duckietop $ source ~/duckietown/set_ros_master.sh duckiebot # your duckiebot
```

On duckiebot, You should also start a the launch
```sh
duckiebot $ roslaunch duckietown_nctu_kaku AGV.launch veh:=duckiebot
```

or

```sh
duckiebot $ roslaunch duckietown_nctu_wama joystick_jupyter.launch veh:=duckiebot
```

If you duckiebot is almost full, you could clean the cache files (~500MB).

```sh
duckiebot $ sudo apt-get clean
```

## Import Packages

In [1]:
import numpy as np
import scipy as sp
import cv2
import time

from matplotlib import pyplot as plt
%matplotlib inline
# set display defaults
plt.rcParams['figure.figsize'] = (10, 10)        # large images
plt.rcParams['image.interpolation'] = 'nearest'  # don't interpolate: show square pixels

## ROS Setup

In [2]:
import sys
# rospy
sys.path.insert(0, '/opt/ros/indigo/lib/python2.7/dist-packages')
# rospkg
sys.path.insert(0, '/usr/lib/python2.7/dist-packages/')

# duckietown_msgs
duckietown_root = '../../'  # this file should be run from {duckietown_root}/turorials/python (otherwise change this line)
sys.path.insert(0, duckietown_root + 'catkin_ws/devel/lib/python2.7/dist-packages')

import rospy
from duckietown_msgs.msg import  Twist2DStamped


### initial a rosnode

In [3]:
rospy.init_node("jupyter_control",anonymous=False)
#please replace "wama" with your duckiebot name
pub_car_cmd = rospy.Publisher("/trabant/car_cmd_switch_node/cmd",Twist2DStamped,queue_size=1)
#rospy.spin()

### define function for publishing car command 

In [4]:
def car_command(v, omega, duration):
# Send stop command
    car_control_msg = Twist2DStamped()
    car_control_msg.v = v 
    car_control_msg.omega = omega 
    pub_car_cmd.publish(car_control_msg)
    rospy.sleep(duration)
    #rospy.loginfo("Shutdown")
    car_control_msg.v = 0.0 
    car_control_msg.omega = 0.0 
    pub_car_cmd.publish(car_control_msg)   


In [5]:
%%bash
rostopic list

/diagnostics
/rosout
/rosout_agg
/trabant/LED_detector_node/switch
/trabant/anti_instagram_node/click
/trabant/anti_instagram_node/corrected_image
/trabant/anti_instagram_node/health
/trabant/anti_instagram_node/transform
/trabant/apriltag_detector_node/switch
/trabant/camera_node/camera_info
/trabant/camera_node/framerate_high_switch
/trabant/camera_node/image/compressed
/trabant/car_cmd_switch_node/cmd
/trabant/decoder_node/switch
/trabant/duckiebot_visualizer/segment_list_markers
/trabant/fsm_node/mode
/trabant/ground_projection/lineseglist_out
/trabant/joy
/trabant/joy_mapper_node/car_cmd
/trabant/joy_mapper_node/joystick_override
/trabant/joy_mapper_node/parallel_autonomy
/trabant/joy_mapper_node/start_avoidance
/trabant/lane_controller_node/car_cmd
/trabant/lane_filter_node/belief_img
/trabant/lane_filter_node/entropy
/trabant/lane_filter_node/in_lane
/trabant/lane_filter_node/lane_pose
/trabant/lane_filter_node/switch
/trabant/lane_pose_visualizer_node/lane_pose_markers
/trabant

## Set State

In [16]:
%%bash
rosservice call /trabant/fsm_node/set_state LANE_FOLLOWING

In [7]:
%%bash
rosservice call /trabant/fsm_node/set_state JOYSTICK_CONTROL

In [10]:
%%bash
rosservice call /trabant/fsm_node/set_state JUPYTER_CONTROL

## EXERCISE: Forward (F), Turn Left (L), or Turn Right (R)

Send commands and calibrate your duckiebot

### Ex1: Forward 0.5 Tile Width

In [11]:
car_command(0.5, 0, 0.75)

### EX2: Turn 45 or 90 Degrees

In [9]:
car_command(0.2, 4, 1.25)

* switch

In [7]:
class switch(object):
    def __init__(self, value):
        self.value = value
        self.fall = False
 
    def __iter__(self):
        """Return the match method once, then stop"""
        yield self.match
        raise StopIteration
     
    def match(self, *args):
        """Indicate whether or not to enter a case suite"""
        if self.fall or not args:
            return True
        elif self.value in args: # changed for v1.5, see below
            self.fall = True
            return True
        else:
            return False

### motion planing lookup function

In [8]:
def motion_planing(concat):
    for i in range(len(concat)):
        primitives = concat[i]
        for case in switch(primitives):
            if case('S'):
                car_command(0.5, 0, 0.6)
                break
            if case('L'):
                car_command(0.2, 4, 0.7)
                break
            if case('R'):
                car_command(0.2, -4, 0.5)
                break
            if case('B'):
                car_command(-0.4, 0, 0.5)
                break

### example: overtaking

In [9]:
overtaking = "LSRSSRSLSS"
motion_planing(overtaking)

### example: parking 

In [12]:
parking = "BBLBBB"
motion_planing(parking)